In [12]:
import pandas as pd

In [34]:
file_path = "data/demographics/csv/Alcova-Heights.csv"
df = pd.read_csv(file_path, skiprows=2)
df

,Number,Percent,Number.1,Percent.1,Unnamed: 4
0,TOTAL POPULATION,"1,900",100.00%,"207,627",100.00%
1,Total Non-Hispanic or Latino,"1,445",76.10%,"176,245",84.90%
2,White alone,"1,105",58.20%,"132,961",64.00%
3,Black or African-American alone,89,4.70%,"17,088",8.20%
4,American Indian or Alaska Native alone,5,0.30%,394,0.20%
5,Asian alone,185,9.70%,"19,762",9.50%
6,Native Hawaiian or Pacific Islander Alone,-,0.00%,133,0.10%
7,Some other Race alone,5,0.30%,611,0.30%
8,Two or more Races,56,2.90%,"5,296",2.60%
9,Total Hispanic or Latino,455,23.90%,"31,382",15.10%


In [35]:
df.drop(df.columns[-2:], axis=1, inplace=True)
df

,Number,Percent,Number.1
0,TOTAL POPULATION,"1,900",100.00%
1,Total Non-Hispanic or Latino,"1,445",76.10%
2,White alone,"1,105",58.20%
3,Black or African-American alone,89,4.70%
4,American Indian or Alaska Native alone,5,0.30%
5,Asian alone,185,9.70%
6,Native Hawaiian or Pacific Islander Alone,-,0.00%
7,Some other Race alone,5,0.30%
8,Two or more Races,56,2.90%
9,Total Hispanic or Latino,455,23.90%


In [36]:
df.columns = ['demographic', 'number', 'percent']
df

,demographic,number,percent
0,TOTAL POPULATION,"1,900",100.00%
1,Total Non-Hispanic or Latino,"1,445",76.10%
2,White alone,"1,105",58.20%
3,Black or African-American alone,89,4.70%
4,American Indian or Alaska Native alone,5,0.30%
5,Asian alone,185,9.70%
6,Native Hawaiian or Pacific Islander Alone,-,0.00%
7,Some other Race alone,5,0.30%
8,Two or more Races,56,2.90%
9,Total Hispanic or Latino,455,23.90%


In [37]:
df.set_index('demographic', drop=True, inplace=True)
df.index.name = None
df

,number,percent
TOTAL POPULATION,"1,900",100.00%
Total Non-Hispanic or Latino,"1,445",76.10%
White alone,"1,105",58.20%
Black or African-American alone,89,4.70%
American Indian or Alaska Native alone,5,0.30%
Asian alone,185,9.70%
Native Hawaiian or Pacific Islander Alone,-,0.00%
Some other Race alone,5,0.30%
Two or more Races,56,2.90%
Total Hispanic or Latino,455,23.90%


In [38]:
df.T

,TOTAL POPULATION,Total Non-Hispanic or Latino,White alone,Black or African-American alone,American Indian or Alaska Native alone,Asian alone,Native Hawaiian or Pacific Islander Alone,Some other Race alone,Two or more Races,Total Hispanic or Latino,...,3-person household,4-person household,5-person household,6-person household,7-or-more-person household,TOTAL HOUSING UNITS,Occupied Housing Units,Owner-Occupied,Renter-Occupied,Vacant Housing Units
number,"1,900","1,445","1,105",89,5,185,-,5,56,455,...,125,79,48,14,19,784,745,546,199,39
percent,100.00%,76.10%,58.20%,4.70%,0.30%,9.70%,0.00%,0.30%,2.90%,23.90%,...,16.80%,10.60%,6.40%,1.90%,2.60%,100.00%,95.00%,69.60%,25.40%,5.00%
